In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import collections
import gzip
import numpy as np
import os
import phate
import time

import pandas as pd
import pickle as pk

from collections import defaultdict
import itertools

# Interactive HTML tools
from ipywidgets import interact
import bokeh
import bokeh.io
from bokeh.io import push_notebook
from bokeh.plotting import figure, show, save, output_notebook, output_file
from bokeh.palettes import Category20b
from bokeh.palettes import Category20
from bokeh.palettes import Category10
from bokeh.palettes import PRGn
from bokeh.palettes import Set1

# Machine-learning and dimensionality reduction tools
import sklearn
from sklearn import decomposition
from sklearn.decomposition import PCA as PCA # We'll use this to check our implementation
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import MDS

import umap

In [ ]:
# Note: PLINK command used to merged data
# sort nhgri_coriell_affy_6_20140825_genotypes.bim > nhgri_coriell_affy_6_20140825_genotypes_sorted.bim
# sort HRS_CLEAR.bim > HRS_CLEAR_sorted.bim

# comm -12 nhgri_coriell_affy_6_20140825_genotypes_sorted.bim HRS_CLEAR_sorted.bim > common_snps_2.txt
# plink --bfile merged_1000G_HRS --pca 200 --out merged_1000G_HRS_pca

data_dir = '/Volumes/Stockage/alex/hrs_1000G/projections'

# Define the files we'll be using
pc_file = 'merged_1000G_HRS_pca.eigenvec'

#aux_path = os.path.join(hrs_data_dir, aux_file)
pc_path = os.path.join(data_dir, pc_file)

# Import auxiliary data. Contains IDs and demographic information.
# NOTE: The auxiliary data is sorted in an order different from the PC data.
#aux_data = []
#with open(aux_path) as input_file:
#    for line in input_file:
#        aux_data.append(line.strip().split(','))

# Import PC data. This data must be converted to an array.
with open(pc_path) as pc:
    pca_contents = pc.readlines()

pca_data = []

for pc in pca_contents:
    pca_data.append(pc.split()[2:len(pc)])

pca_data_array = np.array(pca_data).astype(np.float)

In [ ]:
pca_data_array.shape

# Importing auxiliary data

For labelling we need to import auxiliary dataset from the 1000G and HRS datasets. The 1000G dataset is straightforward.

In [ ]:
# Import 1000G auxiliary data
%store -r continents
%store -r pop_by_continent
%store -r pop
%store -r indices_of_population_members
%store -r name_by_code
%store -r continent_by_population
%store -r individuals
%store -r population_by_individual
%store -r individuals_by_population
%store -r populations
%store -r color_dict

The HRS data needs a bit more work as the variables aren't explicitly defined. We can import the dataset from the HRS notebook and then define values here as needed.

In [ ]:
# Import HRS auxiliary data
%store -r aux_data_hrs
%store -r hisp_dict
%store -r hisp_dict_rev
%store -r mex_dict
%store -r mex_dict_rev
%store -r race_dict
%store -r race_dict_rev
%store -r brn_dict
%store -r brn_dict_rev
%store -r racedb_dict
%store -r racedb_dict_rev

In [ ]:
#aux_data_hrs

In [ ]:
# Labels for the HRS data aren't as clear - import the code to define those as well.
# These auxiliary data sets will define how we label the observations

# Columns are:
# 0 = ID, 1 = Family ID, 2 = Birth Year
# 3 = Hispanic, 4 = Detailed Hispanic, 5 = Race, 6 = Birth Region, 7 = Birth region name
# 10= dbGaP race (Note: Black != AfrAm and White != Not_AfrAm)

# Create multiple types of categorization based on variables to include
aux_data_1 = [] # 1 - Birth region, race, Hispanic status, Mexican status
aux_data_2 = [] # 2 - Race, Hispanic status, Mexican status
aux_data_3 = [] # 3 - Birth region, race
aux_data_4 = [] # 4 - Race, Hispanic status
aux_data_5 = [] # 5 - Birth region
aux_data_6 = [] # 6 - Birth region, Hispanic status, Mexican status
aux_data_7 = [] # 7 - Birth region, Hispanic status

individuals_hrs = []

aux_data_dict_1 = defaultdict(list)
aux_data_dict_2 = defaultdict(list)
aux_data_dict_3 = defaultdict(list)
aux_data_dict_4 = defaultdict(list)
aux_data_dict_5 = defaultdict(list)
aux_data_dict_6 = defaultdict(list)
aux_data_dict_7 = defaultdict(list)

# Get the lists (skip the first row as it's a header)
for a in aux_data_hrs[1:]:
#for a in aux_data_dict[subset][0:]:
    individuals_hrs.append(a[0])
    
    temp_element = [a[0], '_'.join([brn_dict[a[7]], race_dict[a[5]], hisp_dict[a[3]], mex_dict[a[4]]])]    
    aux_data_1.append(temp_element)
    
    temp_element = [a[0], '_'.join([race_dict[a[5]], hisp_dict[a[3]], mex_dict[a[4]]])]
    aux_data_2.append(temp_element)
    
    temp_element = [a[0], '_'.join([brn_dict[a[7]], race_dict[a[5]]])]
    aux_data_3.append(temp_element)
    
    temp_element = [a[0], '_'.join([race_dict[a[5]], hisp_dict[a[3]]])]
    aux_data_4.append(temp_element)
    
    temp_element = [a[0], '_'.join([brn_dict[a[7]]])]
    aux_data_5.append(temp_element)
    
    temp_element = [a[0], '_'.join([brn_dict[a[7]], hisp_dict[a[3]], mex_dict[a[4]]])]
    aux_data_6.append(temp_element)
    
    temp_element = [a[0], '_'.join([brn_dict[a[7]], hisp_dict[a[3]]])]
    aux_data_7.append(temp_element)

In [ ]:
aux_to_use = aux_data_2

In [ ]:
# We must define the population dictionary we wish to use
# The following gives us a collection of all categories of some population and/or proxy for ethnicity:
eth_proxy_set = set([a[1] for a in aux_to_use])
pop_dict = dict()

for e in eth_proxy_set:
    el = e.split('_')
    
    if aux_to_use == aux_data_1:    
        temp_brn = brn_dict_rev[el[0]]
        temp_race = race_dict_rev[el[1]]
        temp_hisp = hisp_dict_rev[el[2]]
        temp_mex = mex_dict_rev[el[3]]
        
        pop_dict.update({e:temp_brn + ' ' + temp_race + ' ' + temp_hisp + ' ' + temp_mex})
    elif aux_to_use == aux_data_2:
        temp_race = race_dict_rev[el[0]]
        temp_hisp = hisp_dict_rev[el[1]]
        temp_mex = mex_dict_rev[el[2]]
        temp_list = [temp_race, temp_hisp, temp_mex]
        
        pop_dict.update({e:temp_race + ' ' + temp_hisp + ' ' + temp_mex})
    elif aux_to_use == aux_data_3:
        temp_brn = brn_dict_rev[el[0]]
        temp_race = race_dict_rev[el[1]]
        
        pop_dict.update({e: temp_brn + ' ' + temp_race})
    elif aux_to_use == aux_data_4:
        temp_race = race_dict_rev[el[0]]
        temp_hisp = hisp_dict_rev[el[1]]
        
        pop_dict.update({e: temp_race + ' ' + temp_hisp})
    elif aux_to_use == aux_data_5:
        temp_brn = brn_dict_rev[el[0]]
        
        pop_dict.update({e: temp_brn})
    elif aux_to_use == aux_data_6:
        temp_brn = brn_dict_rev[el[0]]
        temp_hisp = hisp_dict_rev[el[1]]
        temp_mex = mex_dict_rev[el[2]]
        
        pop_dict.update({e: temp_brn + ' ' + temp_hisp + ' ' + temp_mex})
    elif aux_to_use == aux_data_7:
        temp_brn = brn_dict_rev[el[0]]
        temp_hisp = hisp_dict_rev[el[1]]
        
        pop_dict.update({e: temp_brn + ' ' + temp_hisp})
    
    #print(el, temp_list)
    #print(e.split('_'))

In [ ]:
# Set up an index of each population member and vice versa
# We want to quickly access a given individual's population and a given population's individuals
population_by_individual_hrs = defaultdict(int)
individuals_by_population_hrs = defaultdict(list)

for a in aux_to_use:
    population_by_individual_hrs[a[0]] = a[1]
    individuals_by_population_hrs[a[1]].append(a[0])
    
indices_of_population_members_hrs = defaultdict(list)

for index, indiv in enumerate(individuals_hrs):
    try:
        indices_of_population_members_hrs[population_by_individual_hrs[indiv]].append(index)
    except KeyError:
        continue

# Plotting projections

In [ ]:
# Need to define colour dictionaries - just go with what was previously set up
color_dict_1000g = {}
for i, cont in enumerate(continents): 
    for j, pop in enumerate(pop_by_continent[cont]):
        color_dict_1000g[pop] = Category20b[20][4*i+j%4]

In [ ]:
eth_proxy_set

In [ ]:
%store -r color_dict_born
%store -r color_dict_race_hisp
%store -r color_dict_race_hisp_mex

In [ ]:
color_dict_hrs = color_dict_race_hisp_mex

In [ ]:
# Directory where projection coordinates are stored
proj_dir = '/Volumes/Stockage/alex/hrs_1000G/projections'

# Labels for HRS
if aux_to_use == aux_data_1:
    aux_label = 'BORN_RACE_HISP_MEX'
elif aux_to_use == aux_data_2:
    aux_label = 'RACE_HISP_MEX'
elif aux_to_use == aux_data_3:
    aux_label = 'BORN_RACE'
elif aux_to_use == aux_data_4:
    aux_label = 'RACE_HISP'
elif aux_to_use == aux_data_5:
    aux_label = 'BORN'
elif aux_to_use == aux_data_6:
    aux_label = 'BORN_HISP_MEX'
elif aux_to_use == aux_data_7:
    aux_label = 'BORN_HISP'

# Loop though each tSNE projection
for file in os.listdir(proj_dir):
    num_pcs = file.split('PC_')[1].split('_')[0]
    plex = file.split('PLEX_')[1]
    
    ptitle_str = 'HRS+1000G tSNE on ' + str(num_pcs) + ' PCs, perplexity ' + str(plex)
    fname_str = 'TSNE_HRS_1000G_' + aux_label + '_PC_' + str(num_pcs) + '_PLEX_' + str(plex)
    ftitle_str = 'tSNE on HRS+1000G data, ' + str(num_pcs) + ' PCs, perplexity ' + str(plex)
    arr_str = os.path.join(proj_dir,'HRS_1000G_TSNE_PC_' + str(num_pcs) + '_PLEX_' + str(plex))
    
    int_html_hrs_1000g(ptitle_str, fname_str, ftitle_str, arr_str)
    print('Finished created interactive HTML file on ' + str(num_pcs) + ' PCs with perplexity ' + str(plex))

In [ ]:
# Define a custom function that creates interactive HTML based on file input
# Inputs: Plot title, file name, file title, num_pcs, perplexity, array file location
def int_html_hrs_1000g(ptitle, fname, ftitle, arr):
    component_1_id = 0
    component_2_id = 1

    proj_data = np.loadtxt(arr)

    proj_data_hrs = proj_data[0:(len(aux_data_hrs)-1),:]
    proj_data_1000g = proj_data[len(aux_data_hrs)-1:,:]
    
    p = figure(plot_width=1500, plot_height=1200)
    p.title.text = ptitle

    # First loop is for HRS data
    for pop in sorted(eth_proxy_set):
        proj_pop = proj_data_hrs[indices_of_population_members_hrs[pop]]
        p.circle(proj_pop[:,component_1_id], proj_pop[:,component_2_id], legend=pop_dict[pop],
                 color = color_dict_hrs[pop])

    # Second loop is for 1000G data
    for cont in continents: 
        for pop in pop_by_continent[cont]:
            proj_pop = proj_data_1000g[indices_of_population_members[pop]]
            p.circle(proj_pop[:,component_1_id], proj_pop[:,component_2_id], 
                     legend=name_by_code[pop], color = color_dict[pop])

    p.legend.location = "top_left"

    p.legend.click_policy="hide"

    output_file(fname + '.html',title=ftitle)

    save(p)

def int_html_hrs_1000g_local(ptitle, fname, ftitle, arr):
    component_1_id = 0
    component_2_id = 1

    proj_data = arr

    proj_data_hrs = proj_data[0:(len(aux_data_hrs)-1),:]
    proj_data_1000g = proj_data[len(aux_data_hrs)-1:,:]
    
    p = figure(plot_width=1500, plot_height=1200)
    p.title.text = ptitle

    # First loop is for HRS data
    for pop in sorted(eth_proxy_set):
        proj_pop = proj_data_hrs[indices_of_population_members_hrs[pop]]
        p.circle(proj_pop[:,component_1_id], proj_pop[:,component_2_id], legend=pop_dict[pop],
                 color = color_dict_hrs[pop])

    # Second loop is for 1000G data
    for cont in continents: 
        for pop in pop_by_continent[cont]:
            proj_pop = proj_data_1000g[indices_of_population_members[pop]]
            p.circle(proj_pop[:,component_1_id], proj_pop[:,component_2_id], 
                     legend=name_by_code[pop], color = color_dict[pop])

    p.legend.location = "top_left"

    p.legend.click_policy="hide"

    output_file(fname + '.html',title=ftitle)

    save(p)

In [ ]:
int_html_hrs_1000g_local('test','test','test',mds_proj)

# Create HTML of UMAP projections

In [ ]:
# Labels for HRS
if aux_to_use == aux_data_1:
    aux_label = 'BORN_RACE_HISP_MEX'
elif aux_to_use == aux_data_2:
    aux_label = 'RACE_HISP_MEX'
elif aux_to_use == aux_data_3:
    aux_label = 'BORN_RACE'
elif aux_to_use == aux_data_4:
    aux_label = 'RACE_HISP'
elif aux_to_use == aux_data_5:
    aux_label = 'BORN'
elif aux_to_use == aux_data_6:
    aux_label = 'BORN_HISP_MEX'
elif aux_to_use == aux_data_7:
    aux_label = 'BORN_HISP'

proj_dir = '/Volumes/Stockage/alex/hrs_1000G/projections'
html_dir = '/Volumes/Stockage/alex/hrs_1000G/html'

for fname in os.listdir(proj_dir):
    if os.path.isdir(os.path.join(proj_dir, fname)) or fname=='merged_1000G_HRS_pca.eigenvec':
        continue
    else:
        temp_proj = np.loadtxt(os.path.join(proj_dir, fname))

        num_pcs = fname.split('PC')[1].split('_')[0]
        num_nn = fname.split('NN')[1].split('_')[0]
        md = fname.split('MD')[1].split('_')[0]

        proj_data_hrs = temp_proj[0:(len(aux_data_hrs)-1),:]
        proj_data_1000g = temp_proj[len(aux_data_hrs)-1:,:]

        p = figure(plot_width=1500, plot_height=1200)
        p.title.text = 'PCs: ' + num_pcs + ', NN: ' + num_nn + ', MD: ' + md

        # First loop is for HRS data
        for pop in sorted(eth_proxy_set):
            proj_pop = proj_data_hrs[indices_of_population_members_hrs[pop]]
            p.circle(proj_pop[:,0], proj_pop[:,1], legend=pop_dict[pop],
                     color = color_dict_hrs[pop])

        # Second loop is for 1000G data
        for cont in continents: 
            for pop in pop_by_continent[cont]:
                proj_pop = proj_data_1000g[indices_of_population_members[pop]]
                p.circle(proj_pop[:,0], proj_pop[:,1], 
                         legend=name_by_code[pop], color = color_dict[pop])

        p.legend.location = "top_left"

        p.legend.click_policy="hide"

        output_file(os.path.join(html_dir, fname + '_' + aux_label + '.html'),title='UMAP_'+fname)

        save(p)

In [ ]:
# One-off creations
# Labels for HRS
if aux_to_use == aux_data_1:
    aux_label = 'BORN_RACE_HISP_MEX'
elif aux_to_use == aux_data_2:
    aux_label = 'RACE_HISP_MEX'
elif aux_to_use == aux_data_3:
    aux_label = 'BORN_RACE'
elif aux_to_use == aux_data_4:
    aux_label = 'RACE_HISP'
elif aux_to_use == aux_data_5:
    aux_label = 'BORN'
elif aux_to_use == aux_data_6:
    aux_label = 'BORN_HISP_MEX'
elif aux_to_use == aux_data_7:
    aux_label = 'BORN_HISP'

proj_dir = '/Volumes/Stockage/alex/hrs_1000G/projections'
html_dir = '/Volumes/Stockage/alex/hrs_1000G/html'

fname = 'HRS_1000G_UMAP_PC10_NC2_NN15_MD0.5_2018627203416'

temp_proj = np.loadtxt(os.path.join(proj_dir, fname))

num_pcs = fname.split('PC')[1].split('_')[0]
num_nn = fname.split('NN')[1].split('_')[0]
md = fname.split('MD')[1].split('_')[0]

proj_data_hrs = temp_proj[0:(len(aux_data_hrs)-1),:]
proj_data_1000g = temp_proj[len(aux_data_hrs)-1:,:]

p = figure(plot_width=1500, plot_height=1200)
p.title.text = 'PCs: ' + num_pcs + ', NN: ' + num_nn + ', MD: ' + md

# First loop is for HRS data
for pop in sorted(eth_proxy_set):
    proj_pop = proj_data_hrs[indices_of_population_members_hrs[pop]]
    p.circle(proj_pop[:,0], proj_pop[:,1], legend=pop_dict[pop],
             color = color_dict_hrs[pop])

# Second loop is for 1000G data
for cont in continents: 
    for pop in pop_by_continent[cont]:
        proj_pop = proj_data_1000g[indices_of_population_members[pop]]
        p.circle(proj_pop[:,0], proj_pop[:,1], 
                 legend=name_by_code[pop], color = color_dict[pop])

p.legend.location = "top_left"

p.legend.click_policy="hide"

output_file(os.path.join(html_dir, fname + '_' + aux_label + '.html'),title='UMAP_'+fname)

save(p)

# Create non-interactive images

In [ ]:
# One-off creations
# Labels for HRS
if aux_to_use == aux_data_1:
    aux_label = 'BORN_RACE_HISP_MEX'
elif aux_to_use == aux_data_2:
    aux_label = 'RACE_HISP_MEX'
elif aux_to_use == aux_data_3:
    aux_label = 'BORN_RACE'
elif aux_to_use == aux_data_4:
    aux_label = 'RACE_HISP'
elif aux_to_use == aux_data_5:
    aux_label = 'BORN'
elif aux_to_use == aux_data_6:
    aux_label = 'BORN_HISP_MEX'
elif aux_to_use == aux_data_7:
    aux_label = 'BORN_HISP'

proj_dir = '/Volumes/Stockage/alex/hrs_1000G/projections'
out_dir = '/Volumes/Stockage/alex/hrs_1000G/images'

fname = 'HRS_1000G_UMAP_PC10_NC2_NN15_MD0.5_2018627203416'

temp_proj = np.loadtxt(os.path.join(proj_dir, fname))

num_pcs = fname.split('PC')[1].split('_')[0]
num_nn = fname.split('NN')[1].split('_')[0]
md = fname.split('MD')[1].split('_')[0]

proj_data_hrs = temp_proj[0:(len(aux_data_hrs)-1),:]
proj_data_1000g = temp_proj[len(aux_data_hrs)-1:,:]

#####

fig = plt.figure(figsize=(30,30))
ax = fig.add_subplot(111, aspect=1)

# First loop is for HRS data
for pop in sorted(eth_proxy_set):
    temp_proj = proj_data_hrs[indices_of_population_members_hrs[pop]]
    ax.scatter(temp_proj[:,0], temp_proj[:,1], label=pop_dict[pop], alpha=0.6, color=color_dict_hrs[pop])

# Second loop is for 1000G data
for cont in continents: 
    for pop in pop_by_continent[cont]:
        temp_proj = proj_data_1000g[indices_of_population_members[pop]]
        ax.scatter(temp_proj[:,0], temp_proj[:,1], label=name_by_code[pop], alpha=0.6, color=color_dict[pop])
    
ax.legend(ncol=3,loc='lower center', bbox_to_anchor=(0.55,-0.2), fontsize=12,markerscale=3)

fig.savefig(os.path.join(out_dir, fname + '.jpeg'),format='jpeg')
plt.close()